NOTE: All models and code used here were developed purely for the purpose of illustration. We use a different set of models in production, although the principles are all the same. 

In [45]:
import pandas as pd
import numpy as np
import os
import copy
import dill

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import arviz as az

from plotly.subplots import make_subplots
from cmdstanpy import CmdStanModel

pio.renderers.default = "notebook_connected"

# The Data

The data we are using is publically available [here](https://www.casact.org/publications-research/research/research-resources/loss-reserving-data-pulled-naic-schedule-p). Specifically, we are using the "PP Auto Data Set" (direct download [here](https://www.casact.org/sites/default/files/2021-04/ppauto_pos.csv)). These data include information on private passenger auto policy premiums and losses. The specifics are not too important for our purposes. The important note is that we have data from a number of different auto programs, which makes hierarchical models a natural means to tackle the problem. 

In [46]:
DATA_PATH = os.path.join("..", "..", "data", "ppauto_pos.csv")
MODEL_PATH = os.path.join("..", "..", "stan", "chain_ladder_hierarchical.stan")

Next, we are just doing some preprocessing on the raw data to compute some quantities that we will use later, as well as to remove certain insurance programs from the dataset that have strange values. In reality, we would normally want to be more careful about this step. But for illustration, we are being rather cavelier with removing programs that are "extreme" in some sense. At the end, we are plotting the distributions of "loss ratios" in the data, which is the quantity we will end up modeling:

In [ ]:
pp_auto = pd.read_csv(DATA_PATH)

pp_auto["IncurLoss_B"][pp_auto["IncurLoss_B"]<0] = np.nan
pp_auto["EarnedPremDIR_B"][pp_auto["EarnedPremDIR_B"]<0] = np.nan
pp_auto["loss_ratio"] = (pp_auto["IncurLoss_B"] / pp_auto["EarnedPremDIR_B"]).replace([np.inf, -np.inf], np.nan)
pp_auto["loss_ratio"][pp_auto["loss_ratio"]<=0.0] = np.nan
pp_auto["loss_ratio"][pp_auto["loss_ratio"]>5] = np.nan
pp_auto = pp_auto.dropna(axis=0)

px.histogram(x=pp_auto["loss_ratio"])

The code below just grabs data from an individual program (indicated by `code`) and gives us a few different quantities used in modeling. We are also creating a `loss_obs` variable that we will use for training models, which makes it easy to later look at out-of-sample performance: 

In [48]:
def get_loss_data(df, code):
    df = df.loc[df["GRCODE"] == code]
    loss_true = np.array(pd.pivot(df, index="AccidentYear", columns="DevelopmentLag", values="loss_ratio"))
    premium = np.array(pd.pivot(df, index="AccidentYear", columns="DevelopmentLag", values="EarnedPremDIR_B"))
    AY, DL = loss_true.shape
    loss_obs = copy.copy(loss_true)
    
    for i in range(AY):
        for j in range(DL):
            loss_obs[i,j] = loss_obs[i,j] if i+j < 10 else -1 

    return loss_obs, loss_true, premium, code

In [49]:
train_data = []
test_data = []
premium_data = []
code_data = []

for program_code in pp_auto["GRCODE"].unique()[:30]:
    loss_train, loss_test, premium, code = get_loss_data(pp_auto, program_code)
    if loss_test.shape == loss_train.shape == (10,10):
        if not np.isnan(loss_train).any() and not (loss_train == 0).any():
            train_data.append(loss_train)
            test_data.append(loss_test)
            premium_data.append(premium)
            code_data.append(code)

After iterating through the first 30 programs and including them in our pre-processed data if they meet the criteria specified in the code above, we are left with 25 programs: 

In [ ]:
len(code_data)

Next, we put everything into a dictionary for Stan!

In [51]:
stan_data = {
    "N": len(train_data),
    "AY": train_data[0].shape[0],
    "DL": train_data[0].shape[1],
    "loss": train_data,
    "prior_only": 0,
}

# The Chain-Ladder Model

The Chain-Ladder model below is a simplified variant of the type of model we use at Ledger Investing to "develop" losses to their "ultimate" state. The idea is that, for any given year in which accidents occur, it can take years for us to know what the final (or ultimate) loss ratio (ratio of losses paid out vs premium coming in) will be for the given year (e.g., court cases, adjustments, etc. can take a long time). 

The model below assumes that loss ratios in subsequent "development lags" (i.e. years since looking back at the original accident year) are a function of a free parameter $\text{ATA}_d$ and the previous year's view we had of the loss ratio. 

Then, the the variance is expected to decrease for our view of a given loss ratio as we look at that year further into the future. 

Finally, we assume that the loss ratio for the next development lag will be gamma distributed. The parameter transformations just take our mean+standard deviation parameterization and re-parameterize to the shape+inverse scale parameters of the gamma distribution.

$$
\begin{align*}
    \text{LR}_{t,d} &\sim \Gamma(\alpha_{t,d}, \beta_{t,d}) \\
    \alpha_{t,d} &= \mu_{t,d}^2 / \sigma_{t,d}^2 \\
    \beta_{t,d} &= \mu_{t,d} / \sigma_{t,d}^2 \\
    \mu_{t,d} &= \text{ATA}_d \cdot \text{LR}_{t,d-1} \\
    \sigma_{t,d} &= \exp(\sigma_\text{int} + \sigma_\text{slope} \cdot [d-1])
\end{align*}
$$

$$
\begin{align*}
    \text{LR}_{t,d} &\sim \Gamma(\mu_{t,d}^2 / \sigma_{t,d}^2, \mu_{t,d} / \sigma_{t,d}^2) \\
    \mu_{t,d} &= \text{ATA}_d \cdot \text{LR}_{t,d-1} \\
    \sigma_{t,d} &= \exp(\sigma_\text{int} + \sigma_\text{slope} \cdot [d-1])
\end{align*}
$$

Now, compile and sample!

In [52]:
chain_ladder = CmdStanModel(
    "chain_ladder_hierarchical",
    stan_file=MODEL_PATH,
)

In [ ]:
stan_data["prior_only"] = 0
fit = chain_ladder.sample(
    stan_data,
    iter_warmup=1000, 
    iter_sampling=1000,
    inits=0.2,
    chains=4,
    parallel_chains=4,
    adapt_delta=.8,
    seed=43215,
)

There are some wanrings having to do with exceptions that occur due to the parameter transformations we are making, but overall diagnostics look pretty good: 

In [ ]:
fit.summary()

We can check out the posteriors in more detail:

In [ ]:
az.plot_pair(
    az.from_cmdstanpy(fit),
    var_names=['.*groupmean', '.*groupSD'],
    filter_vars="regex",
    kind='scatter',
    divergences=True,
)

In [ ]:
az.plot_pair(
    az.from_cmdstanpy(fit),
    var_names=['sigma_int', 'sigma_slope'],
    kind='scatter',
    divergences=True,
)

In [ ]:
az.plot_pair(
    az.from_cmdstanpy(fit),
    var_names=['mu_ata_program', 'sigma_ata_program'],
    kind='scatter',
    divergences=True,
)

The function below makes it easy to plot out the posterior predictions against the observed and test data for each accident year:

In [55]:
def plot_predictions(pred_loss, test_data, program_idx):
    fig = make_subplots(2,5)

    x = np.arange(10)
    data_mu = np.nanmean(pred_loss[:,program_idx,:,:], axis=0)
    data_lower = np.nanquantile(pred_loss[:,program_idx,:,:], 0.05, axis=0)
    data_upper = np.nanquantile(pred_loss[:,program_idx,:,:], 0.95, axis=0)

    colors = px.colors.sequential.Viridis

    for i in range(10):
        fig.add_trace(
            go.Scatter(
                x=x, 
                y=data_mu[i], 
                mode="lines",
                marker=dict(color=colors[i]),
                showlegend=False
            ), 
            1 if i<5 else 2, (i % 5) + 1
        )
        fig.add_trace(
            go.Scatter(
                x=x,
                y=data_upper[i],
                mode='lines',
                marker=dict(color=colors[i]),
                line=dict(width=0),
                showlegend=False
            ),
            1 if i<5 else 2, (i % 5) + 1
        )
        fig.add_trace(
            go.Scatter(
                x=x,
                y=data_lower[i],
                mode='lines',
                marker=dict(color=colors[i]),
                line=dict(width=0),
                fill='tonexty',
                showlegend=False
            ),
            1 if i<5 else 2, (i % 5) + 1
        )
        
    for i in range(10):
        row = test_data[program_idx][i]
        row[row==-1] = np.nan
        fig.add_trace(
            go.Scatter(
                x=x, 
                y=row,
                mode='markers',
                name=None,
                showlegend=False,
                marker=dict(color=colors[i], size=10),
            ),
            1 if i<5 else 2, (i % 5) + 1
        )
        fig.add_trace(
            go.Scatter(
                x=x, 
                y=row,
                mode='markers',
                name=None,
                showlegend=False,
                marker=dict(
                    color=colors[i] if i==0 else [0 if train_data[program_idx][i][j] != -1 else 1 for j in range(len(row))], 
                    colorscale=[[0, colors[i]], [1, "white"]]
                ),
            ),
            1 if i<5 else 2, (i % 5) + 1
        )

    return fig

The code below plots the posterior predictions for the program where `index=1`, but you can change the program index to check out performance for different programs: 

In [ ]:
pred_loss = fit.stan_variable(var="pred_loss")
plot_predictions(pred_loss, test_data, 1)

Finally, we will save out the predictions and other information to use in our froecasting model: 

In [112]:
with open(os.path.join("..", "..", "data", "developed_losses.pkl"), "wb") as outfile: 
    dill.dump(pred_loss, outfile)

with open(os.path.join("..", "..", "data", "premium_data.pkl"), "wb") as outfile: 
    dill.dump(premium_data, outfile)

with open(os.path.join("..", "..", "data", "loss_dev_test_data.pkl"), "wb") as outfile: 
    dill.dump(np.array(test_data), outfile)